In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

# Mount Google Drive
drive.mount('/content/drive')

# Load GloVe vectors
def load_glove_embeddings(glove_file):
    embeddings = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    print(f"Loaded {len(embeddings)} word vectors.")
    return embeddings

# Generate sentence embeddings
def generate_sentence_embeddings(sentences, embeddings, embedding_dim=300):
    sentence_vectors = []
    for sentence in sentences:
        tokens = word_tokenize(str(sentence).lower())
        word_vectors = [embeddings[word] for word in tokens if word in embeddings]
        if word_vectors:
            sentence_vector = np.mean(word_vectors, axis=0)
        else:
            sentence_vector = np.zeros(embedding_dim)
        sentence_vector = sentence_vector.flatten()  # Flatten to 1D
        if sentence_vector.shape != (embedding_dim,):
            sentence_vector = np.pad(sentence_vector, (0, embedding_dim - len(sentence_vector)), 'constant')
        sentence_vectors.append(sentence_vector)
    return np.array(sentence_vectors)

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Define individual model functions
def knn_model(X_train, y_train, X_test, y_test):
    model = KNeighborsClassifier()
    model.fit(X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    model = MLPClassifier(max_iter=300)
    model.fit(X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    model = XGBClassifier(eval_metric='mlogloss')
    model.fit(X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    model = AdaBoostClassifier(algorithm='SAMME')
    model.fit(X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    model = CatBoostClassifier(verbose=0)
    model.fit(X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    model = LogisticRegression(max_iter=300)
    model.fit(X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    model = SVC()
    model.fit(X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Define a master function to call all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'
glove_file = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'

# Load the dataset and GloVe embeddings
df = pd.read_excel(input_file)
embeddings = load_glove_embeddings(glove_file)

# Generate GloVe embeddings for the "answer" column
X = generate_sentence_embeddings(df['answer'], embeddings)

# Modify y to make sure the labels start from 0 (shift -1 to 0)
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 400000 word vectors.
Running KNN model...
KNN Model Metrics:
Train Accuracy: 0.7332
Train Precision: 0.7213
Train Recall: 0.7332
Train F1 Score: 0.7247
Train Confusion Matrix:
[[  9  17   6]
 [  5  71  56]
 [  4  39 269]]

Test Accuracy: 0.6488
Test Precision: 0.6684
Test Recall: 0.6488
Test F1 Score: 0.6563
Test Confusion Matrix:
[[  2   5   6]
 [  2  17  22]
 [  5  32 114]]

Running Random Forest model...
Random Forest Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7268
Test Precision: 0.7012
Test Recall: 0.7268
Test F1 Score: 0.6719
Test Confusion Matrix:
[[  1   0  12]
 [  0   7  34]
 [  0  10 141]]

Running Decision Tree model...
Decision Tree Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.993

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Model Metrics:
Train Accuracy: 0.8866
Train Precision: 0.8865
Train Recall: 0.8866
Train F1 Score: 0.8826
Train Confusion Matrix:
[[ 22   8   2]
 [  0  97  35]
 [  1   8 303]]

Test Accuracy: 0.7659
Test Precision: 0.7486
Test Recall: 0.7659
Test F1 Score: 0.7533
Test Confusion Matrix:
[[  2   4   7]
 [  2  21  18]
 [  1  16 134]]

Running XGBoost model...
XGBoost Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7268
Test Precision: 0.6900
Test Recall: 0.7268
Test F1 Score: 0.7005
Test Confusion Matrix:
[[  1   3   9]
 [  1  14  26]
 [  1  16 134]]

Running AdaBoost model...
AdaBoost Model Metrics:
Train Accuracy: 0.7101
Train Precision: 0.6931
Train Recall: 0.7101
Train F1 Score: 0.6815
Train Confusion Matrix:
[[  8   8  16]
 [  0  48  84]
 [  3  27 282]]

Test Accuracy: 0.7122
Test Precision: 0.6646
Test Recall: 0.7122
Test F1 Score: 0.6

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split, RandomizedSearchCV

nltk.download('punkt')

# Mount Google Drive
drive.mount('/content/drive')

# Load GloVe vectors
def load_glove_embeddings(glove_file):
    embeddings = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    print(f"Loaded {len(embeddings)} word vectors.")
    return embeddings

# Generate sentence embeddings
def generate_sentence_embeddings(sentences, embeddings, embedding_dim=300):
    sentence_vectors = []
    for sentence in sentences:
        tokens = word_tokenize(str(sentence).lower())
        word_vectors = [embeddings[word] for word in tokens if word in embeddings]
        if word_vectors:
            sentence_vector = np.mean(word_vectors, axis=0)
        else:
            sentence_vector = np.zeros(embedding_dim)
        sentence_vector = sentence_vector.flatten()  # Flatten to 1D
        if sentence_vector.shape != (embedding_dim,):
            sentence_vector = np.pad(sentence_vector, (0, embedding_dim - len(sentence_vector)), 'constant')
        sentence_vectors.append(sentence_vector)
    return np.array(sentence_vectors)


def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Define a master function to call all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'
glove_file = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'

# Load the dataset and GloVe embeddings
df = pd.read_excel(input_file)
embeddings = load_glove_embeddings(glove_file)

# Generate GloVe embeddings for the "answer" column
X = generate_sentence_embeddings(df['answer'], embeddings)

# Modify y to make sure the labels start from 0 (shift -1 to 0)
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 400000 word vectors.
Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 19}
KNN Model Metrics:
Train Accuracy: 0.7017
Train Precision: 0.6764
Train Recall: 0.7017
Train F1 Score: 0.6445
Train Confusion Matrix:
[[  2  11  19]
 [  1  34  97]
 [  0  14 298]]

Test Accuracy: 0.7073
Test Precision: 0.6221
Test Recall: 0.7073
Test F1 Score: 0.6538
Test Confusion Matrix:
[[  0   1  12]
 [  1   7  33]
 [  0  13 138]]

Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 5, 'max_depth': 30}
Random Forest Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7463
Test Precision: 0.7276
Test Recall: 0.7463
Test F1 Score: 0.6910
Test Confusion Matrix:
[[  1   1  11]
 [  0  

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (50,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.8067
Train Precision: 0.8043
Train Recall: 0.8067
Train F1 Score: 0.7917
Train Confusion Matrix:
[[ 15  10   7]
 [  1  70  61]
 [  1  12 299]]

Test Accuracy: 0.7902
Test Precision: 0.7668
Test Recall: 0.7902
Test F1 Score: 0.7700
Test Confusion Matrix:
[[  3   2   8]
 [  2  18  21]
 [  1   9 141]]

Running XGBoost model...
Best Params: {'subsample': 0.8, 'n_estimators': 100, 'learning_rate': 0.01}
XGBoost Model Metrics:
Train Accuracy: 0.9832
Train Precision: 0.9836
Train Recall: 0.9832
Train F1 Score: 0.9831
Train Confusion Matrix:
[[ 30   0   2]
 [  0 126   6]
 [  0   0 312]]

Test Accuracy: 0.7268
Test Precision: 0.6739
Test Recall: 0.7268
Test F1 Score: 0.6879
Test Confusion Matrix:
[[  1   0  12]
 [  3  10  28]
 [  0  13 138]]

Running AdaBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


...

Best Params: {'learning_rate': 0.3, 'iterations': 150, 'depth': 10}
CatBoost Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7659
Test Precision: 0.7607
Test Recall: 0.7659
Test F1 Score: 0.7312
Test Confusion Matrix:
[[  1   3   9]
 [  0  15  26]
 [  0  10 141]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.7773
Train Precision: 0.7713
Train Recall: 0.7773
Train F1 Score: 0.7645
Train Confusion Matrix:
[[ 12   8  12]
 [  0  73  59]
 [  3  24 285]]

Test Accuracy: 0.7366
Test Precision: 0.7173
Test Recall: 0.7366
Test F1 Score: 0.7239
Test Confusion Matrix:
[[  3   2   8]
 [  1  17  23]
 [  3  17 131]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.7899
Train Precision: 0.8045
Train Recall: 0.7899
Train F1 Score: 0.7607
Train Confusion Matrix:
[[ 10  10  12]
 [  0  57  75]
 [  0   3 309]]

Test Accuracy: 0.7902
Test Precision: 0.7625
Test Recall: 0.7902
Test F1 Score: 0.7550
Test Confusion Matrix:
[[  2   3   8]
 [  1  14  26]
 [  1   4 146]]



In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split, RandomizedSearchCV

nltk.download('punkt')

# Mount Google Drive
drive.mount('/content/drive')

# Load GloVe vectors
def load_glove_embeddings(glove_file):
    embeddings = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    print(f"Loaded {len(embeddings)} word vectors.")
    return embeddings

# Generate sentence embeddings
def generate_sentence_embeddings(sentences, embeddings, embedding_dim=300):
    sentence_vectors = []
    for sentence in sentences:
        tokens = word_tokenize(str(sentence).lower())
        word_vectors = [embeddings[word] for word in tokens if word in embeddings]
        if word_vectors:
            sentence_vector = np.mean(word_vectors, axis=0)
        else:
            sentence_vector = np.zeros(embedding_dim)
        sentence_vector = sentence_vector.flatten()  # Flatten to 1D
        if sentence_vector.shape != (embedding_dim,):
            sentence_vector = np.pad(sentence_vector, (0, embedding_dim - len(sentence_vector)), 'constant')
        sentence_vectors.append(sentence_vector)
    return np.array(sentence_vectors)

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Define a master function to call all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
glove_file = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'

# Load the dataset and GloVe embeddings
df = pd.read_excel(input_file)
embeddings = load_glove_embeddings(glove_file)

# Generate GloVe embeddings for the "answer" column
X = generate_sentence_embeddings(df['cleaned_answer'], embeddings)

# Modify y to make sure the labels start from 0 (shift -1 to 0)
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 400000 word vectors.
Running KNN model...


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Params: {'weights': 'uniform', 'n_neighbors': 18}
KNN Model Metrics:
Train Accuracy: 0.6345
Train Precision: 0.5518
Train Recall: 0.6345
Train F1 Score: 0.5596
Train Confusion Matrix:
[[  1   5  26]
 [  0  16 116]
 [  4  23 285]]

Test Accuracy: 0.6976
Test Precision: 0.5946
Test Recall: 0.6976
Test F1 Score: 0.6334
Test Confusion Matrix:
[[  0   2  11]
 [  0   4  37]
 [  1  11 139]]

Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': None}
Random Forest Model Metrics:
Train Accuracy: 0.9727
Train Precision: 0.9731
Train Recall: 0.9727
Train F1 Score: 0.9724
Train Confusion Matrix:
[[ 30   1   1]
 [  0 122  10]
 [  0   1 311]]

Test Accuracy: 0.7220
Test Precision: 0.6162
Test Recall: 0.7220
Test F1 Score: 0.6469
Test Confusion Matrix:
[[  0   1  12]
 [  0   4  37]
 [  0   7 144]]

Running Decision Tree model...
Best Params: {'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 50}
Decision Tree Model Metrics:
Train Accura

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


...

Best Params: {'learning_rate': 0.3, 'iterations': 50, 'depth': 10}
CatBoost Model Metrics:
Train Accuracy: 0.9748
Train Precision: 0.9754
Train Recall: 0.9748
Train F1 Score: 0.9746
Train Confusion Matrix:
[[ 31   0   1]
 [  0 122  10]
 [  0   1 311]]

Test Accuracy: 0.7171
Test Precision: 0.6271
Test Recall: 0.7171
Test F1 Score: 0.6608
Test Confusion Matrix:
[[  0   3  10]
 [  0   7  34]
 [  0  11 140]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 1}
Logistic Regression Model Metrics:
Train Accuracy: 0.7206
Train Precision: 0.7276
Train Recall: 0.7206
Train F1 Score: 0.6730
Train Confusion Matrix:
[[  6   3  23]
 [  0  38  94]
 [  1  12 299]]

Test Accuracy: 0.7122
Test Precision: 0.6264
Test Recall: 0.7122
Test F1 Score: 0.6572
Test Confusion Matrix:
[[  0   1  12]
 [  1   7  33]
 [  0  12 139]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.8908
Train Precision: 0.8966
Train Recall: 0.8908
Train F1 Score: 0.8856
Train Confusion Matrix:
[[ 25   3   4]
 [  0  91  41]
 [  0   4 308]]

Test Accuracy: 0.6927
Test Precision: 0.6527
Test Recall: 0.6927
Test F1 Score: 0.6687
Test Confusion Matrix:
[[  1   3   9]
 [  2  11  28]
 [  2  19 130]]



In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import nltk
from nltk.tokenize import word_tokenize
from catboost import CatBoostClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, RandomizedSearchCV

nltk.download('punkt')
nltk.download('punkt_tab')

# Mount Google Drive
drive.mount('/content/drive')

# Load GloVe vectors
def load_glove_embeddings(glove_file):
    embeddings = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    print(f"Loaded {len(embeddings)} word vectors.")
    return embeddings

# Generate sentence embeddings
def generate_sentence_embeddings(sentences, embeddings, embedding_dim=300):
    sentence_vectors = []
    for sentence in sentences:
        tokens = word_tokenize(str(sentence).lower())
        word_vectors = [embeddings[word] for word in tokens if word in embeddings]
        if word_vectors:
            sentence_vector = np.mean(word_vectors, axis=0)
        else:
            sentence_vector = np.zeros(embedding_dim)
        sentence_vector = sentence_vector.flatten()  # Flatten to 1D
        if sentence_vector.shape != (embedding_dim,):
            sentence_vector = np.pad(sentence_vector, (0, embedding_dim - len(sentence_vector)), 'constant')
        sentence_vectors.append(sentence_vector)
    return np.array(sentence_vectors)

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Define a master function to call all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
glove_file = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'

# Load the dataset and GloVe embeddings
df = pd.read_excel(input_file)
embeddings = load_glove_embeddings(glove_file)

# Generate GloVe embeddings for the "answer" column
X = generate_sentence_embeddings(df['answer'], embeddings)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(X, variance_threshold=0.95)  # Adjust variance threshold as needed

# Modify y to make sure the labels start from 0 (shift -1 to 0)
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 400000 word vectors.
Number of components selected: 43
Running KNN model...
Best Params: {'weights': 'distance', 'n_neighbors': 16}
KNN Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9937
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   1 311]]

Test Accuracy: 0.6976
Test Precision: 0.6195
Test Recall: 0.6976
Test F1 Score: 0.6511
Test Confusion Matrix:
[[  0   1  12]
 [  1   8  32]
 [  0  16 135]]

Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 30}
Random Forest Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7366
Test Precision: 0.6573
Test Recall: 0.7366
Test F1 Score: 0.6757
Test Confu

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.9265
Train Precision: 0.9273
Train Recall: 0.9265
Train F1 Score: 0.9247
Train Confusion Matrix:
[[ 25   4   3]
 [  0 109  23]
 [  1   4 307]]

Test Accuracy: 0.7415
Test Precision: 0.7220
Test Recall: 0.7415
Test F1 Score: 0.7297
Test Confusion Matrix:
[[  1   3   9]
 [  3  21  17]
 [  1  20 130]]

Running XGBoost model...
Best Params: {'subsample': 0.8, 'n_estimators': 150, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7317
Test Precision: 0.7065
Test Recall: 0.7317
Test F1 Score: 0.7158
Test Confusion Matrix:
[[  2   2   9]
 [  3  16  22]
 [  1  18 132]]

Running AdaBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


...

Best Params: {'learning_rate': 0.3, 'iterations': 150, 'depth': 6}
CatBoost Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7317
Test Precision: 0.6630
Test Recall: 0.7317
Test F1 Score: 0.6907
Test Confusion Matrix:
[[  0   1  12]
 [  0  13  28]
 [  0  14 137]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.7290
Train Precision: 0.7116
Train Recall: 0.7290
Train F1 Score: 0.7020
Train Confusion Matrix:
[[  9   8  15]
 [  2  51  79]
 [  3  22 287]]

Test Accuracy: 0.7317
Test Precision: 0.6971
Test Recall: 0.7317
Test F1 Score: 0.7081
Test Confusion Matrix:
[[  2   1  10]
 [  3  13  25]
 [  3  13 135]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.9034
Train Precision: 0.9077
Train Recall: 0.9034
Train F1 Score: 0.8989
Train Confusion Matrix:
[[ 25   5   2]
 [  0  95  37]
 [  0   2 310]]

Test Accuracy: 0.7171
Test Precision: 0.6895
Test Recall: 0.7171
Test F1 Score: 0.7015
Test Confusion Matrix:
[[  1   3   9]
 [  3  16  22]
 [  2  19 130]]



In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import nltk
from nltk.tokenize import word_tokenize
from catboost import CatBoostClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, RandomizedSearchCV

nltk.download('punkt')
nltk.download('punkt_tab')

# Mount Google Drive
drive.mount('/content/drive')

# Load GloVe vectors
def load_glove_embeddings(glove_file):
    embeddings = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    print(f"Loaded {len(embeddings)} word vectors.")
    return embeddings

# Generate sentence embeddings
def generate_sentence_embeddings(sentences, embeddings, embedding_dim=300):
    sentence_vectors = []
    for sentence in sentences:
        tokens = word_tokenize(str(sentence).lower())
        word_vectors = [embeddings[word] for word in tokens if word in embeddings]
        if word_vectors:
            sentence_vector = np.mean(word_vectors, axis=0)
        else:
            sentence_vector = np.zeros(embedding_dim)
        sentence_vector = sentence_vector.flatten()  # Flatten to 1D
        if sentence_vector.shape != (embedding_dim,):
            sentence_vector = np.pad(sentence_vector, (0, embedding_dim - len(sentence_vector)), 'constant')
        sentence_vectors.append(sentence_vector)
    return np.array(sentence_vectors)

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Define a master function to call all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
glove_file = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'

# Load the dataset and GloVe embeddings
df = pd.read_excel(input_file)
embeddings = load_glove_embeddings(glove_file)

# Generate GloVe embeddings for the "answer" column
X = generate_sentence_embeddings(df['cleaned_answer'], embeddings)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(X, variance_threshold=0.95)  # Adjust variance threshold as needed

# Modify y to make sure the labels start from 0 (shift -1 to 0)
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 400000 word vectors.
Number of components selected: 53
Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 1}
KNN Model Metrics:
Train Accuracy: 0.9748
Train Precision: 0.9749
Train Recall: 0.9748
Train F1 Score: 0.9748
Train Confusion Matrix:
[[ 31   0   1]
 [  0 127   5]
 [  0   6 306]]

Test Accuracy: 0.5707
Test Precision: 0.6243
Test Recall: 0.5707
Test F1 Score: 0.5926
Test Confusion Matrix:
[[  1   4   8]
 [  6  14  21]
 [  6  43 102]]

Running Random Forest model...
Best Params: {'n_estimators': 100, 'min_samples_split': 10, 'max_depth': 10}
Random Forest Model Metrics:
Train Accuracy: 0.9370
Train Precision: 0.9404
Train Recall: 0.9370
Train F1 Score: 0.9318
Train Confusion Matrix:
[[ 16   3  13]
 [  0 119  13]
 [  0   1 311]]

Test Accuracy: 0.7122
Test Precision: 0.5831
Test Recall: 0.7122
Test F1 Score: 0.6303
Test Confu

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (50,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.7668
Train Precision: 0.7592
Train Recall: 0.7668
Train F1 Score: 0.7510
Train Confusion Matrix:
[[ 16   3  13]
 [  3  62  67]
 [  2  23 287]]

Test Accuracy: 0.7317
Test Precision: 0.7125
Test Recall: 0.7317
Test F1 Score: 0.7204
Test Confusion Matrix:
[[  3   3   7]
 [  2  16  23]
 [  5  15 131]]

Running XGBoost model...
Best Params: {'subsample': 0.5, 'n_estimators': 50, 'learning_rate': 0.1}
XGBoost Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9773
Train Recall: 0.9769
Train F1 Score: 0.9767
Train Confusion Matrix:
[[ 31   0   1]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.7366
Test Precision: 0.6585
Test Recall: 0.7366
Test F1 Score: 0.6835
Test Confusion Matrix:
[[  0   2  11]
 [  1   9  31]
 [  0   9 142]]

Running AdaBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


...

Best Params: {'learning_rate': 0.1, 'iterations': 100, 'depth': 6}
CatBoost Model Metrics:
Train Accuracy: 0.9349
Train Precision: 0.9381
Train Recall: 0.9349
Train F1 Score: 0.9303
Train Confusion Matrix:
[[ 17   4  11]
 [  0 117  15]
 [  0   1 311]]

Test Accuracy: 0.7366
Test Precision: 0.6413
Test Recall: 0.7366
Test F1 Score: 0.6561
Test Confusion Matrix:
[[  0   1  12]
 [  0   4  37]
 [  0   4 147]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.7101
Train Precision: 0.6921
Train Recall: 0.7101
Train F1 Score: 0.6745
Train Confusion Matrix:
[[  6   5  21]
 [  2  45  85]
 [  1  24 287]]

Test Accuracy: 0.7122
Test Precision: 0.6513
Test Recall: 0.7122
Test F1 Score: 0.6778
Test Confusion Matrix:
[[  0   2  11]
 [  2  12  27]
 [  0  17 134]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.9286
Train Precision: 0.9316
Train Recall: 0.9286
Train F1 Score: 0.9263
Train Confusion Matrix:
[[ 30   1   1]
 [  0 103  29]
 [  0   3 309]]

Test Accuracy: 0.6732
Test Precision: 0.6416
Test Recall: 0.6732
Test F1 Score: 0.6557
Test Confusion Matrix:
[[  1   5   7]
 [  2  10  29]
 [  6  18 127]]



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.decomposition import PCA

# Function to load GloVe embeddings
def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print(f"Found {len(embeddings_index)} word vectors.")
    return embeddings_index

# Function to process features with GloVe embeddings and numeric features
def process_features_with_glove(input_file, text_column, numeric_features, glove_file):
    # Load the dataset
    df = pd.read_excel(input_file)

    # Load GloVe embeddings
    glove_embeddings = load_glove_embeddings(glove_file)

    # Function to convert text to embedding (averaging word vectors)
    def text_to_embedding(text):
        words = text.split()
        word_vectors = []
        for word in words:
            if word in glove_embeddings:
                word_vectors.append(glove_embeddings[word])
        if len(word_vectors) == 0:
            return np.zeros(glove_embeddings[next(iter(glove_embeddings))].shape[0])  # Return zero vector with the same shape as GloVe embeddings
        return np.mean(word_vectors, axis=0)

    # Convert text column to GloVe embeddings
    embeddings = [text_to_embedding(text) for text in df[text_column]]

    embeddings = np.array(embeddings, dtype=object)

    # Convert the array back to a numeric type assuming all embeddings have the same shape
    embeddings = np.stack(embeddings)

    # Create a DataFrame for GloVe embeddings
    glove_df = pd.DataFrame(embeddings, columns=[f"glove_{i}" for i in range(embeddings.shape[1])])

    # Extract numeric features and combine with GloVe embeddings
    numeric_df = df[numeric_features]
    combined_df = pd.concat([glove_df, numeric_df.reset_index(drop=True)], axis=1)

    return combined_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def main(input_file, text_column, numeric_features, glove_file):
    # Preprocess the dataset
    X, df = process_features_with_glove(input_file, text_column, numeric_features, glove_file)
    y = df['confidence'].apply(lambda x: x + 1)  # Assuming 'confidence' is the target column

    # Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Apply PCA
    X_train_pca, pca = apply_pca(X_train)
    X_test_pca = pca.transform(X_test)

    # Run models
    knn_model(X_train_pca, y_train, X_test_pca, y_test)
    naive_bayes_model(X_train_pca, y_train, X_test_pca, y_test)
    mlp_model(X_train_pca, y_train, X_test_pca, y_test)
    random_forest_model(X_train_pca, y_train, X_test_pca, y_test)
    decision_tree_model(X_train_pca, y_train, X_test_pca, y_test)
    xgboost_model(X_train_pca, y_train, X_test_pca, y_test)
    adaboost_model(X_train_pca, y_train, X_test_pca, y_test)
    catboost_model(X_train_pca, y_train, X_test_pca, y_test)
    logistic_regression_model(X_train_pca, y_train, X_test_pca, y_test)
    svm_model(X_train_pca, y_train, X_test_pca, y_test)

# Call main function with paths
# Input file and feature details
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'answer'
numeric_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']
glove_file = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'
main(input_file,text_column, numeric_features, glove_file)


Found 400000 word vectors.
Number of components selected: 5
Best Params: {'weights': 'uniform', 'n_neighbors': 1}
KNN Model Metrics:
Train Accuracy: 0.9688
Train Precision: 0.9689
Train Recall: 0.9688
Train F1 Score: 0.9687
Train Confusion Matrix:
[[ 29   0   3]
 [  0 139   7]
 [  0   7 359]]

Test Accuracy: 0.5474
Test Precision: 0.5654
Test Recall: 0.5474
Test F1 Score: 0.5546
Test Confusion Matrix:
[[ 0  6  7]
 [ 4  7 16]
 [ 4 25 68]]

Naive Bayes Model Metrics:
Train Accuracy: 0.5772
Train Precision: 0.5201
Train Recall: 0.5772
Train F1 Score: 0.5456
Train Confusion Matrix:
[[  0   3  29]
 [  0  33 113]
 [  0  85 281]]

Test Accuracy: 0.5620
Test Precision: 0.5228
Test Recall: 0.5620
Test F1 Score: 0.5415
Test Confusion Matrix:
[[ 0  2 11]
 [ 0  6 21]
 [ 0 26 71]]



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.7004
Train Precision: 0.6403
Train Recall: 0.7004
Train F1 Score: 0.6280
Train Confusion Matrix:
[[  0   5  27]
 [  0  28 118]
 [  0  13 353]]

Test Accuracy: 0.6788
Test Precision: 0.5698
Test Recall: 0.6788
Test F1 Score: 0.6081
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  4 23]
 [ 0  8 89]]

Best Params: {'n_estimators': 100, 'min_samples_split': 5, 'max_depth': 20}
Random Forest Model Metrics:
Train Accuracy: 0.9632
Train Precision: 0.9637
Train Recall: 0.9632
Train F1 Score: 0.9622
Train Confusion Matrix:
[[ 24   4   4]
 [  0 136  10]
 [  0   2 364]]

Test Accuracy: 0.6350
Test Precision: 0.5417
Test Recall: 0.6350
Test F1 Score: 0.5828
Test Confusion Matrix:
[[ 0  2 11]
 [ 0  4 23]
 [ 0 14 83]]

Best Params: {'min_samples_split': 15, 'min_samples_leaf': 2, 'max_depth': 30}
Decision Tree Model Metrics:
Train Accuracy: 0.8088
Train Precision: 0.7992
Train Recall: 0.8

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'subsample': 1.0, 'n_estimators': 150, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9743
Train Precision: 0.9752
Train Recall: 0.9743
Train F1 Score: 0.9740
Train Confusion Matrix:
[[ 29   0   3]
 [  0 135  11]
 [  0   0 366]]

Test Accuracy: 0.5839
Test Precision: 0.5683
Test Recall: 0.5839
Test F1 Score: 0.5678
Test Confusion Matrix:
[[ 1  3  9]
 [ 1  5 21]
 [ 1 22 74]]



...

Best Params: {'learning_rate': 0.3, 'iterations': 100, 'depth': 4}
CatBoost Model Metrics:
Train Accuracy: 0.8676
Train Precision: 0.8799
Train Recall: 0.8676
Train F1 Score: 0.8552
Train Confusion Matrix:
[[ 11   3  18]
 [  0  99  47]
 [  0   4 362]]

Test Accuracy: 0.6350
Test Precision: 0.5395
Test Recall: 0.6350
Test F1 Score: 0.5809
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  4 23]
 [ 0 14 83]]



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.6728
Train Precision: 0.4527
Train Recall: 0.6728
Train F1 Score: 0.5412
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 146]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 12 is smaller than n_iter=30. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'gamma': 'auto', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.7574
Train Precision: 0.7868
Train Recall: 0.7574
Train F1 Score: 0.7139
Train Confusion Matrix:
[[  8   1  23]
 [  0  45 101]
 [  0   7 359]]

Test Accuracy: 0.6569
Test Precision: 0.5920
Test Recall: 0.6569
Test F1 Score: 0.5983
Test Confusion Matrix:
[[ 1  0 12]
 [ 0  3 24]
 [ 1 10 86]]



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from google.colab import drive
drive.mount('/content/drive')

# Function to load GloVe embeddings
def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print(f"Found {len(embeddings_index)} word vectors.")
    return embeddings_index

# Function to process features with GloVe embeddings and numeric features
def process_features_with_glove(input_file, text_column, numeric_features, glove_file):
    # Load the dataset
    df = pd.read_excel(input_file)

    # Load GloVe embeddings
    glove_embeddings = load_glove_embeddings(glove_file)

    # Function to convert text to embedding (averaging word vectors)
    def text_to_embedding(text):
        # Check if text is a string and not null before splitting
        if isinstance(text, str) and text:
            words = text.split()
            word_vectors = []
            for word in words:
                if word in glove_embeddings:
                    word_vectors.append(glove_embeddings[word])
            if len(word_vectors) == 0:
                return np.zeros(glove_embeddings[next(iter(glove_embeddings))].shape[0])  # Return zero vector with the same shape as GloVe embeddings
            return np.mean(word_vectors, axis=0)
        else:
            # Handle non-string or null values (e.g., return a zero vector)
            return np.zeros(glove_embeddings[next(iter(glove_embeddings))].shape[0])

    # Convert text column to GloVe embeddings
    embeddings = [text_to_embedding(text) for text in df[text_column]]

    embeddings = np.array(embeddings, dtype=object)

    # Convert the array back to a numeric type assuming all embeddings have the same shape
    embeddings = np.stack(embeddings)

    # Create a DataFrame for GloVe embeddings
    glove_df = pd.DataFrame(embeddings, columns=[f"glove_{i}" for i in range(embeddings.shape[1])])

    # Extract numeric features and combine with GloVe embeddings
    numeric_df = df[numeric_features]
    combined_df = pd.concat([glove_df, numeric_df.reset_index(drop=True)], axis=1)

    return combined_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def main(input_file, text_column, numeric_features, glove_file):
    # Preprocess the dataset
    X, df = process_features_with_glove(input_file, text_column, numeric_features, glove_file)
    y = df['confidence'].apply(lambda x: x + 1)  # Assuming 'confidence' is the target column

    # Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Apply PCA
    X_train_pca, pca = apply_pca(X_train)
    X_test_pca = pca.transform(X_test)

    # Run models
    knn_model(X_train_pca, y_train, X_test_pca, y_test)
    naive_bayes_model(X_train_pca, y_train, X_test_pca, y_test)
    mlp_model(X_train_pca, y_train, X_test_pca, y_test)
    random_forest_model(X_train_pca, y_train, X_test_pca, y_test)
    decision_tree_model(X_train_pca, y_train, X_test_pca, y_test)
    xgboost_model(X_train_pca, y_train, X_test_pca, y_test)
    adaboost_model(X_train_pca, y_train, X_test_pca, y_test)
    logistic_regression_model(X_train_pca, y_train, X_test_pca, y_test)
    svm_model(X_train_pca, y_train, X_test_pca, y_test)

# Call main function with paths
# Input file and feature details
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'cleaned_answer'
numeric_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']
glove_file = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'
main(input_file,text_column, numeric_features, glove_file)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 400000 word vectors.
Number of components selected: 14


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'weights': 'uniform', 'n_neighbors': 1}
KNN Model Metrics:
Train Accuracy: 0.9908
Train Precision: 0.9909
Train Recall: 0.9908
Train F1 Score: 0.9908
Train Confusion Matrix:
[[ 32   0   0]
 [  1 142   3]
 [  0   1 365]]

Test Accuracy: 0.5328
Test Precision: 0.5821
Test Recall: 0.5328
Test F1 Score: 0.5452
Test Confusion Matrix:
[[ 1  3  9]
 [ 0 14 13]
 [ 6 33 58]]

Naive Bayes Model Metrics:
Train Accuracy: 0.6581
Train Precision: 0.6022
Train Recall: 0.6581
Train F1 Score: 0.6082
Train Confusion Matrix:
[[  7   5  20]
 [  4  24 118]
 [  8  31 327]]

Test Accuracy: 0.7080
Test Precision: 0.6494
Test Recall: 0.7080
Test F1 Score: 0.6506
Test Confusion Matrix:
[[ 2  1 10]
 [ 1  4 22]
 [ 2  4 91]]



/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.7996
Train Precision: 0.8009
Train Recall: 0.7996
Train F1 Score: 0.7814
Train Confusion Matrix:
[[ 10   6  16]
 [  0  77  69]
 [  1  17 348]]

Test Accuracy: 0.7226
Test Precision: 0.6451
Test Recall: 0.7226
Test F1 Score: 0.6791
Test Confusion Matrix:
[[ 0  2 11]
 [ 1 11 15]
 [ 0  9 88]]

Best Params: {'n_estimators': 150, 'min_samples_split': 5, 'max_depth': 20}
Random Forest Model Metrics:
Train Accuracy: 0.9890
Train Precision: 0.9890
Train Recall: 0.9890
Train F1 Score: 0.9889
Train Confusion Matrix:
[[ 30   1   1]
 [  0 143   3]
 [  0   1 365]]

Test Accuracy: 0.7372
Test Precision: 0.6563
Test Recall: 0.7372
Test F1 Score: 0.6787
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  9 18]
 [ 0  5 92]]

Best Params: {'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 20}
Decision Tree Model Metrics:
Train Accuracy: 0.8529
Train Precision: 0.8492
Train Recall: 0.8

...

Best Params: {'n_estimators': 150, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.7188
Train Precision: 0.7153
Train Recall: 0.7188
Train F1 Score: 0.6633
Train Confusion Matrix:
[[  2   9  21]
 [  0  38 108]
 [  0  15 351]]

Test Accuracy: 0.6861
Test Precision: 0.5827
Test Recall: 0.6861
Test F1 Score: 0.6209
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  5 22]
 [ 0  8 89]]



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 1}
Logistic Regression Model Metrics:
Train Accuracy: 0.6838
Train Precision: 0.6670
Train Recall: 0.6838
Train F1 Score: 0.5908
Train Confusion Matrix:
[[  3   0  29]
 [  0  12 134]
 [  1   8 357]]

Test Accuracy: 0.7007
Test Precision: 0.5714
Test Recall: 0.7007
Test F1 Score: 0.5980
Test Confusion Matrix:
[[ 0  0 13]
 [ 1  1 25]
 [ 0  2 95]]



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 12 is smaller than n_iter=30. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'gamma': 'scale', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.7518
Train Precision: 0.7593
Train Recall: 0.7518
Train F1 Score: 0.7132
Train Confusion Matrix:
[[  8   5  19]
 [  0  48  98]
 [  0  13 353]]

Test Accuracy: 0.7007
Test Precision: 0.6090
Test Recall: 0.7007
Test F1 Score: 0.6384
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  6 21]
 [ 1  6 90]]



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from google.colab import drive
drive.mount('/content/drive')

# Function to load GloVe embeddings
def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print(f"Found {len(embeddings_index)} word vectors.")
    return embeddings_index

# Function to process features with GloVe embeddings and numeric features
def process_features_with_glove(input_file, text_column, numeric_features, glove_file):
    # Load the dataset
    df = pd.read_excel(input_file)

    # Load GloVe embeddings
    glove_embeddings = load_glove_embeddings(glove_file)

    # Function to convert text to embedding (averaging word vectors)
    def text_to_embedding(text):
        # Check if text is a string and not null before splitting
        if isinstance(text, str) and text:
            words = text.split()
            word_vectors = []
            for word in words:
                if word in glove_embeddings:
                    word_vectors.append(glove_embeddings[word])
            if len(word_vectors) == 0:
                return np.zeros(glove_embeddings[next(iter(glove_embeddings))].shape[0])  # Return zero vector with the same shape as GloVe embeddings
            return np.mean(word_vectors, axis=0)
        else:
            # Handle non-string or null values (e.g., return a zero vector)
            return np.zeros(glove_embeddings[next(iter(glove_embeddings))].shape[0])

    # Convert text column to GloVe embeddings
    embeddings = [text_to_embedding(text) for text in df[text_column]]

    # Ensure all embeddings have the same shape before stacking
    embeddings = np.stack(embeddings)

    # Create a DataFrame for GloVe embeddings
    glove_df = pd.DataFrame(embeddings, columns=[f"glove_{i}" for i in range(embeddings.shape[1])])

    # Extract numeric features and combine with GloVe embeddings
    numeric_df = df[numeric_features]
    combined_df = pd.concat([glove_df, numeric_df.reset_index(drop=True)], axis=1)

    # Convert all columns in combined_df to numeric dtype
    for col in combined_df.columns:
        combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')  # 'coerce' will replace invalid values with NaN

    return combined_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def main(input_file, text_column, numeric_features, glove_file):
    # Preprocess the dataset
    X, df = process_features_with_glove(input_file, text_column, numeric_features, glove_file)
    y = df['confidence'].apply(lambda x: x + 1)  # Assuming 'confidence' is the target column

    # Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    # Run models
    knn_model(X_train, y_train, X_test, y_test)
    naive_bayes_model(X_train, y_train, X_test, y_test)
    mlp_model(X_train, y_train, X_test, y_test)
    random_forest_model(X_train, y_train, X_test, y_test)
    decision_tree_model(X_train, y_train, X_test, y_test)
    xgboost_model(X_train, y_train, X_test, y_test)
    adaboost_model(X_train, y_train, X_test, y_test)
    logistic_regression_model(X_train, y_train, X_test, y_test)
    svm_model(X_train, y_train, X_test, y_test)

# Call main function with paths
# Input file and feature details
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'cleaned_answer'
numeric_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']
glove_file = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'
main(input_file,text_column, numeric_features, glove_file)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 400000 word vectors.
Best Params: {'weights': 'distance', 'n_neighbors': 8}
KNN Model Metrics:
Train Accuracy: 0.9908
Train Precision: 0.9909
Train Recall: 0.9908
Train F1 Score: 0.9908
Train Confusion Matrix:
[[ 32   0   0]
 [  1 142   3]
 [  0   1 365]]

Test Accuracy: 0.6131
Test Precision: 0.5197
Test Recall: 0.6131
Test F1 Score: 0.5611
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  3 24]
 [ 0 16 81]]

Naive Bayes Model Metrics:
Train Accuracy: 0.5074
Train Precision: 0.6497
Train Recall: 0.5074
Train F1 Score: 0.5225
Train Confusion Matrix:
[[ 13  11   8]
 [  9 106  31]
 [ 24 185 157]]

Test Accuracy: 0.4745
Test Precision: 0.6318
Test Recall: 0.4745
Test F1 Score: 0.5074
Test Confusion Matrix:
[[ 1  8  4]
 [ 3 17  7]
 [ 6 44 47]]



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (50,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.8971
Train Precision: 0.9010
Train Recall: 0.8971
Train F1 Score: 0.8925
Train Confusion Matrix:
[[ 21   2   9]
 [  1 107  38]
 [  0   6 360]]

Test Accuracy: 0.6788
Test Precision: 0.6199
Test Recall: 0.6788
Test F1 Score: 0.6477
Test Confusion Matrix:
[[ 0  3 10]
 [ 2 10 15]
 [ 1 13 83]]

Best Params: {'n_estimators': 50, 'min_samples_split': 5, 'max_depth': 30}
Random Forest Model Metrics:
Train Accuracy: 0.9798
Train Precision: 0.9799
Train Recall: 0.9798
Train F1 Score: 0.9796
Train Confusion Matrix:
[[ 29   2   1]
 [  0 140   6]
 [  0   2 364]]

Test Accuracy: 0.7591
Test Precision: 0.6883
Test Recall: 0.7591
Test F1 Score: 0.7008
Test Confusion Matrix:
[[ 0  0 13]
 [ 0 10 17]
 [ 0  3 94]]

Best Params: {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None}
Decision Tree Model Metrics:
Train Accuracy: 0.9908
Train Precision: 0.9908
Train Recall: 0.99

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'subsample': 1.0, 'n_estimators': 150, 'learning_rate': 0.01}
XGBoost Model Metrics:
Train Accuracy: 0.9559
Train Precision: 0.9576
Train Recall: 0.9559
Train F1 Score: 0.9550
Train Confusion Matrix:
[[ 27   0   5]
 [  1 128  17]
 [  0   1 365]]

Test Accuracy: 0.7153
Test Precision: 0.6661
Test Recall: 0.7153
Test F1 Score: 0.6630
Test Confusion Matrix:
[[ 0  0 13]
 [ 2  7 18]
 [ 3  3 91]]



...

Best Params: {'n_estimators': 100, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.6893
Train Precision: 0.6889
Train Recall: 0.6893
Train F1 Score: 0.6821
Train Confusion Matrix:
[[ 13   7  12]
 [  1  62  83]
 [  0  66 300]]

Test Accuracy: 0.5912
Test Precision: 0.6133
Test Recall: 0.5912
Test F1 Score: 0.5626
Test Confusion Matrix:
[[ 1  3  9]
 [ 0  3 24]
 [ 0 20 77]]



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 1}
Logistic Regression Model Metrics:
Train Accuracy: 0.7371
Train Precision: 0.7343
Train Recall: 0.7371
Train F1 Score: 0.7023
Train Confusion Matrix:
[[  9   2  21]
 [  0  48  98]
 [  1  21 344]]

Test Accuracy: 0.6934
Test Precision: 0.5996
Test Recall: 0.6934
Test F1 Score: 0.6344
Test Confusion Matrix:
[[ 0  1 12]
 [ 1  6 20]
 [ 0  8 89]]



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 12 is smaller than n_iter=30. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'gamma': 'scale', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.7739
Train Precision: 0.8019
Train Recall: 0.7739
Train F1 Score: 0.7370
Train Confusion Matrix:
[[  8   2  22]
 [  0  53  93]
 [  0   6 360]]

Test Accuracy: 0.6788
Test Precision: 0.5472
Test Recall: 0.6788
Test F1 Score: 0.5946
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  2 25]
 [ 0  6 91]]



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from google.colab import drive
drive.mount('/content/drive')

# Function to load GloVe embeddings
def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print(f"Found {len(embeddings_index)} word vectors.")
    return embeddings_index

# Function to process features with GloVe embeddings and numeric features
def process_features_with_glove(input_file, text_column, numeric_features, glove_file):
    # Load the dataset
    df = pd.read_excel(input_file)

    # Load GloVe embeddings
    glove_embeddings = load_glove_embeddings(glove_file)

    # Function to convert text to embedding (averaging word vectors)
    def text_to_embedding(text):
        # Check if text is a string and not null before splitting
        if isinstance(text, str) and text:
            words = text.split()
            word_vectors = []
            for word in words:
                if word in glove_embeddings:
                    word_vectors.append(glove_embeddings[word])
            if len(word_vectors) == 0:
                return np.zeros(glove_embeddings[next(iter(glove_embeddings))].shape[0])  # Return zero vector with the same shape as GloVe embeddings
            return np.mean(word_vectors, axis=0)
        else:
            # Handle non-string or null values (e.g., return a zero vector)
            return np.zeros(glove_embeddings[next(iter(glove_embeddings))].shape[0])

    # Convert text column to GloVe embeddings
    embeddings = [text_to_embedding(text) for text in df[text_column]]

    # Ensure all embeddings have the same shape before stacking
    embeddings = np.stack(embeddings)

    # Create a DataFrame for GloVe embeddings
    glove_df = pd.DataFrame(embeddings, columns=[f"glove_{i}" for i in range(embeddings.shape[1])])

    # Extract numeric features and combine with GloVe embeddings
    numeric_df = df[numeric_features]
    combined_df = pd.concat([glove_df, numeric_df.reset_index(drop=True)], axis=1)

    # Convert all columns in combined_df to numeric dtype
    for col in combined_df.columns:
        combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')  # 'coerce' will replace invalid values with NaN

    return combined_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def main(input_file, text_column, numeric_features, glove_file):
    # Preprocess the dataset
    X, df = process_features_with_glove(input_file, text_column, numeric_features, glove_file)
    y = df['confidence'].apply(lambda x: x + 1)  # Assuming 'confidence' is the target column

    # Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    # Run models
    knn_model(X_train, y_train, X_test, y_test)
    naive_bayes_model(X_train, y_train, X_test, y_test)
    mlp_model(X_train, y_train, X_test, y_test)
    random_forest_model(X_train, y_train, X_test, y_test)
    decision_tree_model(X_train, y_train, X_test, y_test)
    xgboost_model(X_train, y_train, X_test, y_test)
    adaboost_model(X_train, y_train, X_test, y_test)
    logistic_regression_model(X_train, y_train, X_test, y_test)
    svm_model(X_train, y_train, X_test, y_test)

# Call main function with paths
# Input file and feature details
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'answer'
numeric_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']
glove_file = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'
main(input_file,text_column, numeric_features, glove_file)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 400000 word vectors.
Best Params: {'weights': 'uniform', 'n_neighbors': 1}
KNN Model Metrics:
Train Accuracy: 0.9743
Train Precision: 0.9743
Train Recall: 0.9743
Train F1 Score: 0.9741
Train Confusion Matrix:
[[ 29   2   1]
 [  0 139   7]
 [  0   4 362]]

Test Accuracy: 0.6058
Test Precision: 0.5681
Test Recall: 0.6058
Test F1 Score: 0.5863
Test Confusion Matrix:
[[ 0  3 10]
 [ 1  8 18]
 [ 3 19 75]]

Naive Bayes Model Metrics:
Train Accuracy: 0.4430
Train Precision: 0.6391
Train Recall: 0.4430
Train F1 Score: 0.4701
Train Confusion Matrix:
[[ 21   5   6]
 [ 35  84  27]
 [ 60 170 136]]

Test Accuracy: 0.4088
Test Precision: 0.5773
Test Recall: 0.4088
Test F1 Score: 0.4455
Test Confusion Matrix:
[[ 2  5  6]
 [ 4 15  8]
 [16 42 39]]



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.8713
Train Precision: 0.8716
Train Recall: 0.8713
Train F1 Score: 0.8670
Train Confusion Matrix:
[[ 24   3   5]
 [  0 100  46]
 [  0  16 350]]

Test Accuracy: 0.7226
Test Precision: 0.7060
Test Recall: 0.7226
Test F1 Score: 0.6984
Test Confusion Matrix:
[[ 1  5  7]
 [ 1 13 13]
 [ 0 12 85]]

Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': None}
Random Forest Model Metrics:
Train Accuracy: 0.9743
Train Precision: 0.9747
Train Recall: 0.9743
Train F1 Score: 0.9741
Train Confusion Matrix:
[[ 29   0   3]
 [  0 137   9]
 [  0   2 364]]

Test Accuracy: 0.6642
Test Precision: 0.5356
Test Recall: 0.6642
Test F1 Score: 0.5839
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  2 25]
 [ 0  8 89]]

Best Params: {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 10}
Decision Tree Model Metrics:
Train Accuracy: 0.9559
Train Precision: 0.9566
Train Recall: 0.9

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'subsample': 1.0, 'n_estimators': 50, 'learning_rate': 0.1}
XGBoost Model Metrics:
Train Accuracy: 0.9669
Train Precision: 0.9685
Train Recall: 0.9669
Train F1 Score: 0.9664
Train Confusion Matrix:
[[ 29   0   3]
 [  0 131  15]
 [  0   0 366]]

Test Accuracy: 0.7007
Test Precision: 0.6645
Test Recall: 0.7007
Test F1 Score: 0.6668
Test Confusion Matrix:
[[ 2  0 11]
 [ 1  8 18]
 [ 1 10 86]]



...

Best Params: {'n_estimators': 100, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.7298
Train Precision: 0.7112
Train Recall: 0.7298
Train F1 Score: 0.6971
Train Confusion Matrix:
[[ 13   5  14]
 [  2  44 100]
 [  1  25 340]]

Test Accuracy: 0.6861
Test Precision: 0.6653
Test Recall: 0.6861
Test F1 Score: 0.6594
Test Confusion Matrix:
[[ 1  1 11]
 [ 1 11 15]
 [ 0 15 82]]



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 1}
Logistic Regression Model Metrics:
Train Accuracy: 0.7224
Train Precision: 0.7369
Train Recall: 0.7224
Train F1 Score: 0.6593
Train Confusion Matrix:
[[  4   4  24]
 [  0  31 115]
 [  0   8 358]]

Test Accuracy: 0.6934
Test Precision: 0.6245
Test Recall: 0.6934
Test F1 Score: 0.6234
Test Confusion Matrix:
[[ 1  0 12]
 [ 1  3 23]
 [ 0  6 91]]



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 12 is smaller than n_iter=30. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'linear', 'gamma': 'scale', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.7923
Train Precision: 0.7893
Train Recall: 0.7923
Train F1 Score: 0.7764
Train Confusion Matrix:
[[ 20   1  11]
 [  2  68  76]
 [  4  19 343]]

Test Accuracy: 0.6277
Test Precision: 0.5824
Test Recall: 0.6277
Test F1 Score: 0.6015
Test Confusion Matrix:
[[ 2  2  9]
 [ 3  4 20]
 [ 5 12 80]]

